In [1]:
import ml
reload(ml)
from ml import *
import timeit
import scipy
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
from datetime import datetime
from collections import Counter
import matplotlib.pyplot as plt
from pprint import pprint
from __future__ import division
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
sns.set_style('whitegrid')
plt.style.use('seaborn-whitegrid')
%matplotlib inline

__author__ = 'HK Dambanemuya'
__version__ = 'Python 2'

'''
    Analysis originaly performed in Python 2 (deprecated)
    Seaborn, Statsmodel, and * imports broken in Python 3
'''

In [2]:
listing_data = pd.read_csv("../Data/prosper_final.csv")
listing_data.head()

,ListingKey,FirstBidCreationDate,LastBidCreationDate,NoBids,AmountRequested,AvgInterBidTime,Gini,DebtToIncomeRatio,DescriptionLength,RepaidOrNot,...,ProsperRating,Friend_Count,Endorsement_Count,Role_Count,Group_Count,Watching,MemberCreationDate,TimeToFirstBid,TimeBetweenFirstLastBid,BorrowerAge
0,00033425227988088FA6752,2008-06-21 22:38:29,2008-06-23 06:08:09,15,1000.000,3101.733,0.167,0.190,190,1,...,0,1,1,3,3,3,2008-06-12 16:44:38,188.717,449.667,9
1,7B343428727859426204CF7,2008-08-05 19:49:08,2008-08-09 13:22:12,14,2200.000,22880.643,0.539,0.190,383,1,...,0,1,1,3,3,3,2008-06-12 16:44:38,75.767,1053.067,54
2,000433785890431972B4743,2007-01-03 09:11:14,2007-01-05 14:05:06,30,2550.000,8520.767,0.298,0.100,577,0,...,0,1,0,1,3,3,2006-05-10 16:13:00,1180.600,293.867,236
3,00083422661625108817246,2008-06-05 22:22:35,2008-06-12 22:03:28,141,3500.000,10633.589,0.198,0.350,210,1,...,0,1,0,1,3,3,2008-05-26 10:52:14,636.233,1420.883,0
4,000A34209897973969CFA81,2008-05-09 17:31:57,2008-05-15 15:17:26,2,1500.000,47348.000,0.468,0.040,134,1,...,0,1,0,2,3,3,2008-05-07 11:04:48,138.267,1305.483,1


In [3]:
listing_data.columns

Index([u'ListingKey', u'FirstBidCreationDate', u'LastBidCreationDate',
       u'NoBids', u'AmountRequested', u'AvgInterBidTime', u'Gini',
       u'DebtToIncomeRatio', u'DescriptionLength', u'RepaidOrNot',
       u'ListingCreationDate', u'ListingStartDate', u'MemberKey', u'Category',
       u'BidMaximumRate', u'BidCount', u'ProsperPrincipalBorrowed',
       u'ProsperPrincipalOutstanding', u'Duration', u'Images',
       u'IsBorrowerHomeowner', u'ActiveProsperLoans', u'BorrowerRate',
       u'BorrowerMaximumRate', u'TotalProsperLoans', u'ProsperScore',
       u'GroupLeaderRewardRate', u'EffectiveYield', u'FundingOption',
       u'EstimatedReturn', u'EstimatedLoss', u'MonthlyLoanPayment',
       u'OnTimeProsperPayments', u'ProsperPaymentsLessThanOneMonthLate',
       u'ProsperPaymentsOneMonthPlusLate', u'TotalProsperPaymentsBilled',
       u'BankDraftFeeAnnualRate', u'CreditScoreRange', u'NumImages',
       u'LenderYieldRate', u'LenderAvgWinningBids', u'LenderAvgTotalBids',
       u'Lender

In [5]:
bid_data = pd.read_csv("../Data/bid_notick.txt", sep="|")
bid_data.head()

,Amount,CreationDate,Bid_Key,ListingKey,ListingStatus,MemberKey,MinimumRate,MinimumYield,ParticipationAmount,Status,ModifiedDate,Unnamed: 11
0,50.000,2006-02-21 20:23:20,0F12336634427919902FE93,D6473365888221963456102,Completed,F4273365023760472DBD934,0.250,0.245,0.000,Outbid,2006-02-21 20:23:20,nan
1,50.000,2006-02-21 22:50:00,87F333655285187004B2EE8,D6473365888221963456102,Completed,F4273365023760472DBD934,nan,nan,50.000,Winning,2006-02-21 22:50:00,nan
2,50.000,2006-02-28 00:16:01,9DB53365565361098646F41,3B09336533376614186EEF7,Expired,F4273365023760472DBD934,0.045,0.040,0.000,Outbid,2006-02-28 00:16:01,nan
3,50.000,2006-02-28 20:06:43,7284336544796747049D125,601833660859080305A2AC3,Completed,F4273365023760472DBD934,0.069,0.064,0.000,Outbid,2006-02-28 20:06:43,nan
4,50.000,2006-03-01 09:27:50,F3FD3365245307569E84B9F,601833660859080305A2AC3,Completed,F4273365023760472DBD934,nan,nan,50.000,Winning,2006-03-01 09:27:50,nan


In [6]:
bid_data.columns

Index([u'Amount', u'CreationDate', u'Bid_Key', u'ListingKey', u'ListingStatus',
       u'MemberKey', u'MinimumRate', u'MinimumYield', u'ParticipationAmount',
       u'Status', u'ModifiedDate', u'Unnamed: 11'],
      dtype='object')

## Data Description

In [18]:
print "Start Date: ", np.min(listing_data.ListingStartDate)
print "End Date: ", np.max(listing_data.ListingStartDate)
print "Borrower Count: ", listing_data.MemberKey.nunique()
print "Total Listings: ", bid_data.ListingKey.nunique()
print "Funded Listings: ", listing_data.ListingKey.nunique()
print "Repaid Listings: ", Counter(listing_data.RepaidOrNot).get(1)
print "Defaulted Listings: ", Counter(listing_data.RepaidOrNot).get(0)
print "Minimum Amount Requested: $", np.min(listing_data.AmountRequested)
print "Median Amount Requested: $", np.median(listing_data.AmountRequested)
print "Maximum Amount Requested: $", np.max(listing_data.AmountRequested)
print "Minimum Bid Amount: $", np.min(bid_data.Amount)
print "Median Bid Amount: $", np.median(bid_data.Amount)
print "Maximum Bid Amount: $", np.max(bid_data.Amount)
print "Listing Status: ", Counter(bid_data[["ListingKey", "ListingStatus"]].groupby("ListingKey").agg('max')["ListingStatus"]).items()[0]

Start Date:  2005-11-09 20:44:28
End Date:  2008-10-12 21:47:23
Borrower Count:  26404
Funded Listings:  28934
Repaid Listings:  18279
Defaulted Listings:  10656
Minimum Amount Requested: $ 1000.0
Median Amount Requested: $ 4500.0
Maximum Amount Requested: $ 25000.0
Minimum Bid Amount: $ -50.099999999999994
Median Bid Amount: $ 50.0
Maximum Bid Amount: $ 24999.999999999996
Listing Status:  ('Completed', 51696)


In [8]:
bid_data.MemberKey.nunique()

65756

In [16]:
Counter(bid.data.ListingStatus)

[('Completed', 51696), ('Active', 433), ('Cancelled', 11406), ('Pending Completion', 223), ('Expired', 132589), ('Withdrawn', 40062)]
